In [1]:
import numpy as np
import math
from dataclasses import dataclass
from typing import List, Dict
import pandas as pd
from copy import deepcopy
from collections import defaultdict

# Обновленные глобальные константы
TOTAL_RESIDENTS = 1200    # Уменьшено с 10000
TOTAL_WORKPLACES = 5000   # Уменьшено с 11250

@dataclass
class District:
    id: int
    type: str
    coords: tuple
    residents: Dict[str, int]
    workplaces: Dict[str, int]
    salaries: Dict[str, float]

@dataclass
class Workplace:
    id: int
    district_id: int
    skill: str
    specialization: int
    salary: float
    occupied: bool = False

@dataclass
class Agent:
    id: int
    district_id: int
    skill: str
    specialization: int
    salary_exp: float
    alpha: List[float]

def initialize_districts() -> List[District]:
    districts = []
    district_id = 0
    a = 1/6

    # Центральные районы (4)
    central_salaries = {'low': round(2*a,4), 'medium': round(4*a,4), 'high': round(6*a,4)}
    residents = {
        'low': math.floor(TOTAL_RESIDENTS * 0.10 * 0.10),   # 10% населения, 70% low
        'medium': math.floor(TOTAL_RESIDENTS * 0.10 * 0.20), 
        'high': math.floor(TOTAL_RESIDENTS * 0.10 * 0.70)
    }
    workplaces = {
        'low': math.floor(TOTAL_WORKPLACES * 0.30 * 0.70),
        'medium': math.floor(TOTAL_WORKPLACES * 0.30 * 0.20),
        'high': math.floor(TOTAL_WORKPLACES * 0.30 * 0.10)
    }
    for coord in [(0,0.125), (0,-0.125), (0.125,0), (-0.125,0)]:
        districts.append(District(
            id=district_id, type='central', coords=coord,
            residents=residents.copy(),
            workplaces=workplaces.copy(),
            salaries=central_salaries.copy()))
        district_id +=1

    # Смешанные районы (8)
    mixed_salaries = {'low': round(1.5*a,4), 'medium': round(3*a,4), 'high': round(4.5*a,4)}
    residents = {
        'low': math.floor(TOTAL_RESIDENTS * 0.30 * 0.50),
        'medium': math.floor(TOTAL_RESIDENTS * 0.30 * 0.30),
        'high': math.floor(TOTAL_RESIDENTS * 0.30 * 0.20)
    }
    workplaces = {
        'low': math.floor(TOTAL_WORKPLACES * 0.50 * 0.60),
        'medium': math.floor(TOTAL_WORKPLACES * 0.50 * 0.25),
        'high': math.floor(TOTAL_WORKPLACES * 0.50 * 0.15)
    }
    for coord in [(0,0.25),(0,-0.25),(0.25,0),(-0.25,0),
                  (0.2,0.15),(-0.2,0.15),(0.2,-0.15),(-0.2,-0.15)]:
        districts.append(District(
            id=district_id, type='mixed', coords=coord,
            residents=residents.copy(),
            workplaces=workplaces.copy(),
            salaries=mixed_salaries.copy()))
        district_id +=1

    # Спальные районы (8)
    sleep_salaries = {'low':round(a,4), 'medium':round(2*a,4), 'high':round(3*a,4)}
    residents = {
        'low': math.floor(TOTAL_RESIDENTS * 0.60 * 0.80),
        'medium': math.floor(TOTAL_RESIDENTS * 0.60 * 0.15),
        'high': math.floor(TOTAL_RESIDENTS * 0.60 * 0.05)
    }
    workplaces = {
        'low': math.floor(TOTAL_WORKPLACES * 0.20 * 0.80),
        'medium': math.floor(TOTAL_WORKPLACES * 0.20 * 0.10),
        'high': math.floor(TOTAL_WORKPLACES * 0.20 * 0.10)
    }
    for coord in [(0,0.5),(0,-0.5),(0.5,0),(-0.5,0),
                  (0.3,0.4),(0.3,-0.4),(-0.3,0.4),(-0.3,-0.4)]:
        districts.append(District(
            id=district_id, type='sleep', coords=coord,
            residents=residents.copy(),
            workplaces=workplaces.copy(),
            salaries=sleep_salaries.copy()))
        district_id +=1
    
    return districts

def initialize_workplaces(districts: List[District]) -> List[Workplace]:
    workplaces = []
    wp_id = 0
    for d in districts:
        for skill in ['low', 'medium', 'high']:
            for _ in range(d.workplaces[skill]):
                workplaces.append(Workplace(
                    id=wp_id, district_id=d.id, skill=skill,
                    specialization=np.random.randint(1,6),
                    salary=d.salaries[skill]
                ))
                wp_id +=1
    return workplaces
# Модель с алгоритмом Гейла-Шепли
class PreferenceModelWithGS:
    def __init__(self, districts, workplaces):
        self.districts = districts
        self.workplaces = workplaces
        self.distance_matrix = self._calc_distance_matrix()
        self.alphas = [
            [0.7, 0.1, 0.1, 0.1],
            [0.1, 0.7, 0.1, 0.1],
            [0.1, 0.1, 0.7, 0.1],
            [0.2, 0.1, 0.1, 0.7]
        ]
        self.skill_levels = ['low', 'medium', 'high']
        self.skill_index = {s: i for i, s in enumerate(self.skill_levels)}

    def _calc_distance_matrix(self):
        return np.array([
            [math.hypot(d1.coords[0] - d2.coords[0], d1.coords[1] - d2.coords[1])
                for d2 in self.districts]
            for d1 in self.districts
        ])

    def _generate_agents(self):
        agents = []
        agent_id = 0
        for d in self.districts:
            for skill in self.skill_levels:
                for _ in range(d.residents[skill]):
                    alpha = self.alphas[agent_id % 4]
                    agents.append(Agent(
                        id=agent_id,
                        district_id=d.id,
                        skill=skill,
                        specialization=(agent_id % 5) + 1,
                        salary_exp=d.salaries[skill],
                        alpha=alpha
                    ))
                    agent_id += 1
        return agents

    def _agent_utility(self, agent: Agent, wp: Workplace) -> float:
        # Агент не может работать на месте с более высокой квалификацией
        if self.skill_index[agent.skill] < self.skill_index[wp.skill]:
            return float('inf')

        H1 = 0 if agent.specialization == wp.specialization else 1
        H2 = (self.skill_index[agent.skill] - self.skill_index[wp.skill]) / 2
        H3 = abs(agent.salary_exp - wp.salary) / agent.salary_exp
        H4 = self.distance_matrix[agent.district_id][wp.district_id]

        return sum(a * h for a, h in zip(agent.alpha, [H1, H2, H3, H4]))

    def _gale_shapley(self, agents: List[Agent], workplaces: List[Workplace]):
        agent_prefs = {}
        wp_prefs = {}

        agent_map = {a.id: a for a in agents}
        wp_map = {w.id: w for w in workplaces}

        # Список рабочих мест, допустимых по квалификации
        for agent in agents:
            valid_wps = [
                wp.id for wp in workplaces
                if self.skill_index[agent.skill] >= self.skill_index[wp.skill]
            ]
            agent_prefs[agent.id] = sorted(
                valid_wps,
                key=lambda wp_id: self._agent_utility(agent, wp_map[wp_id])
            )

        for wp in workplaces:
            suitable_agents = [
                a.id for a in agents
                if self.skill_index[a.skill] >= self.skill_index[wp.skill]
            ]
            wp_prefs[wp.id] = sorted(
                suitable_agents,
                key=lambda a_id: (
                    0 if agent_map[a_id].specialization == wp.specialization else 1,
                    self._agent_utility(agent_map[a_id], wp)
                )
            )

        matches = {}
        proposals = {a.id: 0 for a in agents}
        free_agents = set(agent_prefs.keys())

        while free_agents:
            agent_id = free_agents.pop()
            prefs = agent_prefs[agent_id]

            # Агент может не иметь ни одного подходящего рабочего места
            if proposals[agent_id] >= len(prefs):
                continue

            wp_id = prefs[proposals[agent_id]]
            proposals[agent_id] += 1

            if wp_id not in matches:
                matches[wp_id] = agent_id
                wp_map[wp_id].occupied = True
            else:
                current_agent_id = matches[wp_id]
                wp_rank = wp_prefs[wp_id]
                if wp_rank.index(agent_id) < wp_rank.index(current_agent_id):
                    matches[wp_id] = agent_id
                    free_agents.add(current_agent_id)
                else:
                    free_agents.add(agent_id)

            # Если агент ещё не исчерпал список, он остаётся свободным
            if proposals[agent_id] < len(prefs) and agent_id not in matches.values():
                free_agents.add(agent_id)

        return matches

    def run(self):
        agents = self._generate_agents()
        workplaces = deepcopy(self.workplaces)
        np.random.shuffle(agents)

        matches = self._gale_shapley(agents, workplaces)
        agent_dict = {a.id: a for a in agents}

        matrix = np.zeros((len(self.districts), len(self.districts)))
        matched_agents = set()

        for wp_id, agent_id in matches.items():
            agent = agent_dict.get(agent_id)
            wp = next(w for w in workplaces if w.id == wp_id)
            if agent:
                matrix[agent.district_id, wp.district_id] += 1
                matched_agents.add(agent.id)

        print(f"✅ Сопоставлено: {len(matched_agents)} / {len(agents)} агентов")
        return matrix
  
# Визуализация
def print_matrix(matrix, districts):
    df = pd.DataFrame(matrix,
                     index=[f"{d.type}-{d.id}" for d in districts],
                     columns=[f"{d.type}-{d.id}" for d in districts])
    print(df.round().astype(int))

if __name__ == "__main__":
    districts = initialize_districts()
    workplaces = initialize_workplaces(districts)
    
    model = PreferenceModelWithGS(districts, workplaces)
    matrix = model.run()
    matrix = matrix/100
    print("\nМатрица корреспонденций:")
    print_matrix(matrix, districts)


KeyboardInterrupt: 

In [27]:
matrix.sum()

np.float64(91.19999999999999)

In [25]:
matrix1 = matrix/100
matrix1

array([[0.0099, 0.    , 0.    , 0.    , 0.0019, 0.    , 0.    , 0.    ,
        0.0002, 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    ],
       [0.    , 0.0102, 0.    , 0.    , 0.    , 0.0018, 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    ],
       [0.0001, 0.    , 0.0103, 0.    , 0.    , 0.    , 0.0016, 0.    ,
        0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    ],
       [0.0001, 0.    , 0.    , 0.0103, 0.    , 0.    , 0.    , 0.0016,
        0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , 0.    , 0.036 , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , 0.    , 0.    , 0.036 , 0.    , 0.    ,
   

In [26]:
matrix1.sum()

np.float64(0.9119999999999999)